### dataprep.ipynb
Creates .csv file of prediction data.

Final version of dataprep.ipynb.

In [1]:
# import libraries
import pandas as pd
import numpy as np
from datetime import datetime
from enum import Enum

'''enum class(es)'''
'''======================================================='''
class TeamGameType(Enum):
    HOME = "home",
    VISITOR = "visitor",
    BOTH = "both"

''' prepare + clean dataframe '''
'''======================================================='''

# read file
data = pd.read_csv('../data/more_games_data.csv');

# parse dates
data['GAME_DATE_EST'] = pd.to_datetime(data['GAME_DATE_EST'], format="%d/%m/%Y")



Remove rows with missing data or unrealistic/outlying values

In [2]:
# drop rows with empty data, drop unneeded columns

print("No. of rows before removing outliers: ", data.shape[0]);

# get column to re-insert later
data_sb_total_col = data.loc[:,"SB_TOTAL"]

drop_cols = ['SEASON', 'OVER_CONSENSUS_%', 'UNDER_CONSENSUS_%', 'OVER_CONSENSUS', 'UNDER_CONSENSUS', 'SB_TOTAL'];

data = data.drop(drop_cols, axis='columns');

# drop remaining rows with NaN
data = data.dropna()

# re-insert with possible NaN values
data.insert(3, 'SB_TOTAL', data_sb_total_col)

# drop outlying data
# keep rows where scorebook total is null or between 75-300
data = data[(data['SB_TOTAL'].isna()) | ((data['SB_TOTAL'] <= 300) & (data['SB_TOTAL'] >= 75))]

# keep rows where points are in a realistic range
data = data.loc[(data['PTS_home'] <= 200) | (data['PTS_home'] >= 20)]
data = data.loc[(data['PTS_away'] <= 200) | (data['PTS_away'] >= 20)]

print("No. of rows after removing outliers: ", data.shape[0]);

data

No. of rows before removing outliers:  5498
No. of rows after removing outliers:  5496


,GAME_ID,GAME_STATUS_TEXT,GAME_DATE_EST,SB_TOTAL,HOME_TEAM_ABR,VISITOR_TEAM_ABR,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,22200477,Final,2022-12-22,228.0,NOP,SAS,126,0.484,0.926,0.382,25,46,117,0.478,0.815,0.321,23,45,1
1,22200478,Final,2022-12-22,232.0,UTA,WAS,120,0.488,0.952,0.457,16,40,112,0.561,0.765,0.333,20,38,1
2,22200466,Final,2022-12-21,215.5,CLE,MIL,114,0.482,0.786,0.313,22,37,106,0.470,0.682,0.433,20,47,1
3,22200467,Final,2022-12-21,222.0,PHI,DET,113,0.441,0.909,0.297,27,49,93,0.392,0.735,0.261,15,47,1
4,22200468,Final,2022-12-21,235.5,ATL,CHI,108,0.429,0.000,0.378,22,47,110,0.500,0.773,0.292,20,48,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5493,41200305,Final,2013-05-30,NaN,MIA,IND,90,0.507,0.583,0.389,19,32,79,0.449,0.800,0.385,11,33,1
5494,41200304,Final,2013-05-28,NaN,IND,MIA,99,0.500,0.788,0.214,18,49,92,0.390,0.889,0.348,15,30,1
5495,41200314,Final,2013-05-27,NaN,MEM,SAS,86,0.372,0.708,0.357,19,41,93,0.513,0.923,0.231,23,34,0
5496,41200303,Final,2013-05-26,NaN,IND,MIA,96,0.397,0.682,0.571,16,45,114,0.545,0.857,0.429,21,36,0


Remove outlying data by removing rows with values over 4 standard deviations from the mean of all values in their column. Some columns were excluded from this

In [3]:
from scipy.stats import zscore

print("No. of rows before removing outliers: ", data.shape[0]);

data = data[(np.abs(zscore(data.loc[:, ((data.columns != 'GAME_DATE_EST') & \
                                       (data.columns != 'GAME_STATUS_TEXT') & \
                                       (data.columns != 'GAME_ID') & \
                                       (data.columns != 'HOME_TEAM_ABR') & \
                                       (data.columns != 'VISITOR_TEAM_ABR') & \
                                       (data.columns != 'SB_TOTAL'))].values)) < 4).all(axis=1)]

print("No. of rows after removing outliers: ", data.shape[0]);

No. of rows before removing outliers:  5496
No. of rows after removing outliers:  5416


In [4]:
''' define useful functions '''
'''======================================================='''

# get all games played by a given team (home or away games)
# params: team is string abbreviation of the team; game_type is enum to select home games, is away games, or both 
def get_games_with_team(team, game_type):
    team_home_games = data.loc[data['HOME_TEAM_ABR'] == team]
    team_away_games = data.loc[data['VISITOR_TEAM_ABR'] == team]
    
    match game_type:
        case TeamGameType.HOME:
            return team_home_games.sort_values(by='GAME_DATE_EST', ascending=False).reset_index(drop=True);
        case TeamGameType.VISITOR:
            return team_away_games.sort_values(by='GAME_DATE_EST', ascending=False).reset_index(drop=True);
        case TeamGameType.BOTH:
            team_all_games = pd.merge(team_home_games, team_away_games, how='outer').sort_values(by='GAME_DATE_EST', ascending=False)
            return team_all_games.reset_index(drop=True);

# get any (home, away, or both) of last amount (n) of games for a given team
# if n is too big, max amount of games available will be returned
# default is to get all games (BOTH)
def get_previous_n_games(date, team, n, game_type = TeamGameType.BOTH):
    last_game = get_previous_game(date, team, game_type);
    
    if (last_game.shape[0] > 0):
        last_game_index = last_game.index[0];

        previous_n_df = data[data.index > last_game_index];

        # remove rows after index + n
        previous_n_df = previous_n_df[previous_n_df.index < last_game_index + (n + 1)];
    else:
        previous_n_df = pd.DataFrame();
    
    return previous_n_df;

# get first game played by a team that appears in 'data' dataframe on or before specified date
# starts from most recent date working backwards in time
# get either previous home game, away game or any (default)
def get_previous_game(date, team, game_type = TeamGameType.BOTH):
    #team_games = get_games_with_team(team, game_type);
    if (game_type == TeamGameType.BOTH):
        game_on_date = data[(data['GAME_DATE_EST'] <= date) & \
                            (data["HOME_TEAM_ABR"] == team | data["VISITOR_TEAM_ABR"] == team)];
    elif (game_type == TeamGameType.HOME):
        game_on_date = data[(data['GAME_DATE_EST'] <= date) & (data["HOME_TEAM_ABR"] == team)];
    elif (game_type == TeamGameType.VISITOR):
        game_on_date = data[(data['GAME_DATE_EST'] <= date) & (data["VISITOR_TEAM_ABR"] == team)];
        
    game_on_date = game_on_date.iloc[:1];        
    return game_on_date;

# calculate average win rate of dataset
# game_type_home - whether the team played home or away
def calculate_win_rate(n_games_df, game_type_home):
    i = 0;
    win_rate = 0;
    for x in n_games_df.index:
        # if played at home count home win
        # else count away win
        if (game_type_home == True):
            if (n_games_df.loc[x, "HOME_TEAM_WINS"] == 1):
                win_rate = win_rate + 1;
        else:
            if (n_games_df.loc[x, "HOME_TEAM_WINS"] == 0):
                win_rate = win_rate + 1;
        i = i + 1;
    win_rate = win_rate / i;
    return win_rate;


In [5]:
''' form and export new dataframe '''
'''======================================================='''
new_rows = [];
i=0;
for x in data.index:
    
    # n amount of a team's last games will be used
    n_maximum = 25;
    n_minimum = 5;
    home_team_abbreviation = data.loc[x, "HOME_TEAM_ABR"];
    away_team_abbreviation = data.loc[x, "VISITOR_TEAM_ABR"];

    # get past 'n' home and away games for home team
    h_home_last_n_df = get_previous_n_games(data.loc[x, "GAME_DATE_EST"], home_team_abbreviation, n_maximum, TeamGameType.HOME);
    h_away_last_n_df = get_previous_n_games(data.loc[x, "GAME_DATE_EST"], home_team_abbreviation, n_maximum, TeamGameType.VISITOR);
    
    # combine dataframes and get average values 
    if (h_home_last_n_df.shape[0] >= n_minimum and h_away_last_n_df.shape[0] >= n_minimum):
        h_both_last_n_df = pd.concat([h_home_last_n_df, h_away_last_n_df], axis=0)
        home_avgs_array = h_home_last_n_df[["PTS_home", "FT_PCT_home", "FG_PCT_home", "FG3_PCT_home", \
                                               "AST_home", "REB_home"]].mean(skipna=True);
        h_win_rate = calculate_win_rate(h_home_last_n_df, True);
        
    # do the same for away team
    a_home_last_n_df = get_previous_n_games(data.loc[x, "GAME_DATE_EST"], away_team_abbreviation, n_maximum, TeamGameType.HOME);
    a_away_last_n_df = get_previous_n_games(data.loc[x, "GAME_DATE_EST"], away_team_abbreviation, n_maximum, TeamGameType.VISITOR);

    if (a_home_last_n_df.shape[0] >= n_minimum and a_away_last_n_df.shape[0] >= n_minimum):
        a_both_last_n_df = pd.concat([a_home_last_n_df, a_away_last_n_df], axis=0)
        away_avgs_array = a_away_last_n_df[["PTS_home", "FT_PCT_home", "FG_PCT_home", "FG3_PCT_home", \
                                               "AST_home", "REB_home"]].mean(skipna=True);
        a_win_rate = calculate_win_rate(a_away_last_n_df, False);
        
    actual_total = data.loc[x, "PTS_home"] + data.loc[x, "PTS_away"]
    
    sb_total = data.loc[x, 'SB_TOTAL']
    
    # create new row with average data of last n games
    new_row_data = data.loc[x, "GAME_DATE_EST"], home_team_abbreviation, away_team_abbreviation, \
    home_avgs_array[0], away_avgs_array[0], h_win_rate, a_win_rate, \
    home_avgs_array[1], away_avgs_array[1], home_avgs_array[2], away_avgs_array[2], \
    home_avgs_array[3], away_avgs_array[3], home_avgs_array[4], away_avgs_array[4], \
    home_avgs_array[5], away_avgs_array[5], sb_total, actual_total;
        
    # append row to list of new rows
    # if actual total is equal to sb total, dont use row
    new_rows.append(new_row_data);
    
# append rows to dataframe
output_df = pd.DataFrame(new_rows,
                           columns = ["DATE", "HOME_TEAM_ABR", "AWAY_TEAM_ABR", "HOME_PTS_AVG", "AWAY_PTS_AVG", \
                                      "HOME_WIN_RATE_AVG", "AWAY_WIN_RATE_AVG", "HOME_FT_PCT", "AWAY_FT_PCT", \
                                      "HOME_FG_PCT", "AWAY_FG_PCT", "HOME_FG3_PCT", "AWAY_FG3_PCT", \
                                      "HOME_REB_AVG", "AWAY_REB_AVG", "HOME_AST_AVG", "AWAY_AST_AVG", \
                                      "PTS_TOTAL_SCOREBOOK", "PTS_TOTAL"
                                     ]);

In [6]:
#export to .csv file
output_df.to_csv('../output_files/dataprep_output.csv', encoding='utf-8', index=False)